# PC2135: Thermodynamics and Statistical Physics - Mini Project I

## Setup

We consider 5 ideal gas particles constrained within a 3D box of dimensions $L \times L \times L$, having energy $200E_0$, where

$$
\begin{equation}
    E_0 = \frac{\hbar^2}{2m}\frac{\pi}{L^2}
\end{equation}
$$

Since the particles exist in a 3D box, the Schrödinger equation results in a wave equation with 3 degrees of freedom &mdash; 3 $n$'s required to completely specify the state of each particle. From there, we can get that the energy of the $i$-th particle will be

$$
\begin{equation}
\tag{2}
    E_i = (n_{x_i}^2 + n_{y_i}^2 + n_{z_i}^2)E_0
\end{equation}
$$

Hence, summing over the energies of all five particles, we must get $200E_0$ which implies

$$
\begin{equation}
\tag{3}
    \sum_{i=1}^5 (n_{x_i}^2 + n_{y_i}^2 + n_{z_i}^2) = 200
\end{equation}
$$

By finding the set of all 15-tuple of $n$'s that satisfy to equation (3), we will then be able to know the full set of microstates that result in the measured macroscopic energy level (i.e. all _accessible_ states). Only then can we perform further calculations on the probability of states.

Our approach hence first focuses on solving the above equation.

## Enumerating accessible states

To solve equation (3) which has 15 unknowns, our approach is simple:

1. Fix the number of the first $n$, with maximum given such that $n_{max}^2 < 200$
2. Subtract $n^2$ from the energy threshold: $200-n^2$
3. Recursively solve for the remaining degrees of freedom, updating the threshold for each call.

However, we will need to filter for _impossible_ configurations, which means that when the recursion reaches the last degree of freedom, there exists no positive integer $n$ that fulfills $n^2 = E_{threshold}$.

Thus, we will need to filter for these microstates, and this can be done by simply checking whether the last $E_{threshold}$ (i.e. energy remaining for the last degree of freedom) is a perfect square.
- If yes, then we include that configuration into the accesible states
- If no, then we omit the previous $n$'s found

All the valid configurations will then be added to a list, where the function `get_microstates` will then return this list of all accessible states.

In [111]:
import numpy as np
import numpy.ma as ma
import math

dimension = 3
E_total = 200 # multiples of E0
n_particles = 5  # number of particles


def get_unique_microstates(n_E: int, num_particles: int):
    states = get_configuration(n_E, num_particles * dimension)
    mask = np.where(states[:,0] != 0)

    valid_states = states[mask]
    return np.flip(valid_states, axis = 1)

def get_configuration(n_E: int, degrees: int, last_n: int = 1):
    max_n = int(np.ceil(np.sqrt(n_E)))
    microstates = np.empty(shape=[0, degrees])

    is_square = (max_n * max_n == n_E)

    if degrees==1:
        if is_square and max_n>=last_n:
            return np.array([[max_n]])
        else:
            return np.array([[0]])

    for i in range(last_n, max_n):
        sub_states = get_configuration(n_E - i**2, degrees - 1, i)
        l = len(sub_states)
        a = np.empty(shape=[l,1])
        a.fill(i)

        sub_states = np.append(sub_states, a, axis = 1)

        for state in sub_states:
            microstates = np.append(microstates, [state], axis = 0)

    return microstates

def get_number_permutations(specific_microstate: np.ndarray):
    uniques, count = np.unique(specific_microstate, axis = 0, return_counts=True)
    l = len(specific_microstate)
    
    ans = math.factorial(l)
    for i in count:
        ans = ans/math.factorial(i)
        
    return ans

def get_total_permutations(microstates: np.ndarray):
    counter = 0
    for state in microstates:
        counter += get_number_permutations(state)
    
    return counter

states = get_unique_microstates(E_total, n_particles)
print(states)
print(get_total_permutations(states))

[[ 1.  1.  1. ...  2.  9. 10.]
 [ 1.  1.  1. ...  4.  5. 12.]
 [ 1.  1.  1. ...  6.  7. 10.]
 ...
 [ 2.  3.  3. ...  4.  5.  7.]
 [ 2.  3.  3. ...  4.  4.  4.]
 [ 3.  3.  3. ...  4.  4.  5.]]
14543054580.0


In [112]:
def find_min(microstates: np.ndarray):
    first_3 = microstates[:, :dimension]
    sums = [np.sum(arr * arr) for arr in first_3]
    min = np.min(sums)
    mins = np.where(sums == min)
    uniques = np.unique(first_3[mins], axis=0)
    return (uniques, min)

min, min_e = find_min(states)
print(min)
print(min_e)

[[1. 1. 1.]]
3.0


In [113]:
def enumerate_min(microstates: np.ndarray):
    mins = find_min(microstates)[0]
    counter = 0

    for min in mins:
        min_mask = np.where((microstates[:, :dimension] == min).all(axis=1), True, False)
        min_states = microstates[min_mask]
        counter += get_total_permutations(min_states[:, dimension:]) * get_total_permutations([min])
        
    return counter

print(enumerate_min(states))

169580664.0


In [114]:
def find_max(microstates: np.ndarray):
    last_3 = microstates[:, -dimension:]
    sums = [np.sum(arr * arr) for arr in last_3]
    max = np.max(sums)
    maxes = np.where(sums == max)
    uniques = np.unique(last_3[maxes], axis = 0)
    return (uniques, max)

max, max_e = find_max(states)
print(max)
print(max_e)

[[ 2.  9. 10.]
 [ 4.  5. 12.]
 [ 6.  7. 10.]]
185.0


In [115]:
def enumerate_max(microstates: np.ndarray):
    maxes = find_max(microstates)[0]
    counter = 0

    for max in maxes:
        max_mask = np.where((microstates[:, -dimension:] == max).all(axis=1), True, False)
        max_states = microstates[max_mask]

        counter += get_total_permutations(max_states[:, :-dimension]) * get_total_permutations([max])

    return counter

print(enumerate_max(states))

216.0


In [116]:
# code below has not been updated

def enumerate_microstates(microstates: list[tuple]):
    return len(microstates)


def calc_prob_min_energy(microstates: list[tuple]):
    return enumerate_min(microstates) / enumerate_microstates(microstates)


def calc_prob_max_energy(microstates: list[tuple]):
    return enumerate_max(microstates) / enumerate_microstates(microstates)


print(states)
print(enumerate_microstates(states))
print(enumerate_min(states))
print(enumerate_max(states))
print(calc_prob_min_energy(states))
print(calc_prob_max_energy(states))

[[ 1.  1.  1. ...  2.  9. 10.]
 [ 1.  1.  1. ...  4.  5. 12.]
 [ 1.  1.  1. ...  6.  7. 10.]
 ...
 [ 2.  3.  3. ...  4.  5.  7.]
 [ 2.  3.  3. ...  4.  4.  4.]
 [ 3.  3.  3. ...  4.  4.  5.]]
351
169580664.0
216.0
483135.7948717949
0.6153846153846154


(a) the lowest possible energy (in terms of E0) possible for the first particle.
(b) the probability of that particle having said energy.
(c) the highest energy (in terms of E0) possible for the first particle.
(d) the probability of that particle having said energy.